<a href="https://colab.research.google.com/github/matheusses/PdfAssinadorDigital/blob/master/DFS_BFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DFS - Deep First Search
BFS - Breath First Search

In [2]:
import copy

In [4]:
# Question: Given the above parameters, find the conversion rate that maps to the 'from' currency to the 'to' currency from every single query. Your return value should be a number.

# Example: You are given the following parameters:

# Rates: ['USD', 'JPY', 100] ['JPY', 'CHN', 20] ['CHN', 'THAI', 200]
# Queries: [['USD', 'CHN'], ['JPY', 'THAI'], ['USD', 'AUD']]
# Output: [1000.0, 4000.0, -1.0]


# inputString = "AUD:USD:0.90 USD:BRL:0.16 EUR:USD:1.10 CAD:USD:1.1"

# criar uma funcao que receba um pair de moedas e retorne o FX ration


# 1
# simple hash table

# T/S = O(n)
def convertStringToDicRatios(ratiosString: str) -> dict:
# add each pair into the hash table
    parts = ratiosString.split(" ")
    ratiosHashtable = {}
    for part in parts:
        ratioParts = part.split(":")

        if(ratioParts[0] not in ratiosHashtable): ratiosHashtable[ratioParts[0]] = {}
        if(ratioParts[1] not in ratiosHashtable): ratiosHashtable[ratioParts[1]] = {}

        ratiosHashtable[ratioParts[0]][ratioParts[1]] = float(ratioParts[2])
        ratiosHashtable[ratioParts[1]][ratioParts[0]] = 1/float(ratioParts[2])

    return ratiosHashtable

# T/S = O(1)
def getFxRatioBFS(ratiosHashtable: {}, currencyOrigin: str, currencyTarget: str) -> float:
    queue, visited = deque(), set()
    queue.append((currencyOrigin, 1, f"{currencyOrigin}"))
    visited.add(currencyOrigin)

    while queue:
        current, ratio, path = queue.popleft()
        print(current, ratio, path)
        if currencyTarget == current: return ratio

        for nextCurrency in ratiosHashtable[current]:
            if nextCurrency not in visited:
                nextRatio = ratiosHashtable[current][nextCurrency]
                queue.append((nextCurrency, ratio * nextRatio, f"{path}->{nextCurrency}"))
                visited.add(nextCurrency)
    return -1

def getFxRatioDfs(ratiosHashtable: {}, currencyOrigin: str, currencyTarget: str) -> float:
    print("*******", currencyOrigin, currencyTarget)
    return DfsSearch(ratiosHashtable, currencyOrigin, currencyTarget, 1, set(), currencyOrigin)

def DfsSearch(ratiosHashtable: {}, current: str, currencyTarget: str, ratio: int, visited: set, path: str):
    visited.add(current)
    print(currencyTarget, path)

    if currencyTarget == current: return round(ratio, 2)

    for nextCurrency in ratiosHashtable[current]:
        if nextCurrency not in visited:
            nextRatio = ratiosHashtable[current][nextCurrency]
            ret = DfsSearch(ratiosHashtable, nextCurrency, currencyTarget, ratio * nextRatio, copy.deepcopy(visited), f"{path}->{nextCurrency}")
            if ret:
                return ret

    return None


# Tests
inputString = "AUD:USD:0.90 EUR:BRD:3 AUD:CAD:1.13 CAD:BRL:0.127433 USD:BRL:0.16 BRL:JPY:1.2 EUR:USD:1.10 CAD:USD:1.20 BRL:NOK:1.4 BRL:CHN:2"
ratiosHashtable = convertStringToDicRatios(inputString)

# ("AUD", "USD") -> ("USD", "BRL")

### 2 case: "AUD -> JPY" -->> ("AUD", "USD") -> ("USD", "BRL") -> ("JPY", "BRL")

# Define test cases and expected results
test_cases = [
    (("AUD", "USD"), 0.90),
    (("USD", "BRL"), 0.16),
    (("EUR", "USD"), 1.10),
    (("USD", "EUR"), 1/1.10),
    (("CAD", "USD"), 0.80),
    (("USD", "CAD"), 1/0.796),
    (("EUR", "JPY"), 1.10 * 0.16 * 1.2),
    (("AUD", "BRL"), 0.90 * 0.16)
]

# Run tests using a for loop
for args, expected in test_cases:
    result = getFxRatioDfs(ratiosHashtable, *args)
    assert result == round(expected, 2), f"❌ {args}: {result} != {expected}"
    print(f"✅ {args}: {result} == {expected}")



******* AUD USD
USD AUD
USD AUD->USD
✅ ('AUD', 'USD'): 0.9 == 0.9
******* USD BRL
BRL USD
BRL USD->AUD
BRL USD->AUD->CAD
BRL USD->AUD->CAD->BRL
✅ ('USD', 'BRL'): 0.16 == 0.16
******* EUR USD
USD EUR
USD EUR->BRD
USD EUR->USD
✅ ('EUR', 'USD'): 1.1 == 1.1
******* USD EUR
EUR USD
EUR USD->AUD
EUR USD->AUD->CAD
EUR USD->AUD->CAD->BRL
EUR USD->AUD->CAD->BRL->JPY
EUR USD->AUD->CAD->BRL->NOK
EUR USD->AUD->CAD->BRL->CHN
EUR USD->BRL
EUR USD->BRL->CAD
EUR USD->BRL->CAD->AUD
EUR USD->BRL->JPY
EUR USD->BRL->NOK
EUR USD->BRL->CHN
EUR USD->EUR
✅ ('USD', 'EUR'): 0.91 == 0.9090909090909091
******* CAD USD
USD CAD
USD CAD->AUD
USD CAD->AUD->USD
✅ ('CAD', 'USD'): 0.8 == 0.8
******* USD CAD
CAD USD
CAD USD->AUD
CAD USD->AUD->CAD
✅ ('USD', 'CAD'): 1.26 == 1.2562814070351758
******* EUR JPY
JPY EUR
JPY EUR->BRD
JPY EUR->USD
JPY EUR->USD->AUD
JPY EUR->USD->AUD->CAD
JPY EUR->USD->AUD->CAD->BRL
JPY EUR->USD->AUD->CAD->BRL->JPY
✅ ('EUR', 'JPY'): 0.21 == 0.21120000000000003
******* AUD BRL
BRL AUD
BRL AUD->USD